# SQL

### <a id='the_destination_0'></a> Оглавление
- <a href='#the_destination_1'>1. Загрузка данных</a>
- <a href='#the_destination_2'>2. Решение задач</a>
- <a href='#the_destination_3'>3. Выводы</a>

Цель проекта - ознакомится с базой нашего нового интернет-магазина/книжной платформы, выявить критические, либо наоборот самые интересные для бизнеса направления.

Наша задача — проанализировать базу данных крупного сервиса для чтения книг. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценное предложение для нового продукта.

Первым делом загрузим данные и подготовим их для работы.

Далее выделим самых активных издателей, самые высокие оценки среди авторов, посмотрим на активность пользователей и рассчитаем общее количество книг а также в отношении к дате выхода.

## <a id='the_destination_1'></a> Загрузка данных
<a href='#the_destination_0'>К оглавлению</a>

In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student',
             'pwd': 'Sdf4$2;d-d30pp',
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,
             'db': 'data-analyst-final-project-db'}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

<b>Все отлично!👍:</b> Библиотеки импортировали, коннектор сделали - отлично!
    
Спасибо, что перезапускаешь ядро проекта перед отправкой на ревью - это важный шаг, который поможет избежать простых и досадных ошибок при воспроизведении решнеия:)
</div>

Посмотрим на данные, предварительно задав функцию, которая выведет 5 первых строк каждого датафрейма.

In [3]:
list = ['books', 'authors', 'publishers', 'reviews']
for name in list:
    print('\t')
    query = "SELECT * FROM {} LIMIT 5".format(name)
    print('Первые 5 строк из датасета', name, ':')
    display(pd.io.sql.read_sql(query, con = engine))

	
Первые 5 строк из датасета books :


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


	
Первые 5 строк из датасета authors :


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


	
Первые 5 строк из датасета publishers :


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


	
Первые 5 строк из датасета reviews :


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Описание данных:**

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## <a id='the_destination_2'></a> Решение задач
<a href='#the_destination_0'>К оглавлению</a>

**1 задание:** Посчитайте, сколько книг вышло после 1 января 2000 года.

In [4]:
query = '''
            SELECT COUNT(DISTINCT(book_id))
            FROM books
            WHERE publication_date >= '2000-01-01'
        '''
pd.io.sql.read_sql(query, con = engine) 

,count
0,821


C 1 января 2000 года вышла 821 книга.

**2 задание:** Для каждой книги посчитайте количество обзоров и среднюю оценку.

In [5]:
query = '''
            SELECT b.book_id, 
                   b.title, 
                   COUNT(DISTINCT re.review_id) AS cnt_reviews, 
                   ROUND(AVG(rt.rating),3) AS avg_rating
            FROM books b
            LEFT JOIN reviews re ON b.book_id = re.book_id
            LEFT JOIN ratings rt ON b.book_id = rt.book_id
            GROUP BY 1, 2
            ORDER BY 3 DESC
        '''
pd.io.sql.read_sql(query, con = engine) 

,book_id,title,cnt_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.663
1,963,Water for Elephants,6,3.977
2,734,The Glass Castle,6,4.207
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.415
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.667
996,808,The Natural Way to Draw,0,3.000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000
998,221,Essential Tales and Poems,0,4.000


Самое большое количество обзоров получила книга из серии "Сумерки" - 7 отзывов, хотя рейтинг не высокий. Так же в данных много книг вообще без обзоров.

**3 задание:** Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

In [6]:
query = '''
            SELECT p.publisher,
                   COUNT(DISTINCT b.book_id) as cnt_books
            FROM books b
            LEFT JOIN publishers p ON b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY 1
            ORDER BY 2 DESC
            LIMIT 1            
        '''
pd.io.sql.read_sql(query, con = engine) 

,publisher,cnt_books
0,Penguin Books,42


Исключив из данных брошюры (книги меньше 50 страниц), получили, что издательство "Penguin Books" выпустило больше всех книг, которые есть в нашей базе, а именно 42 шт.

**4 задание:** Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

In [7]:
query = '''
            WITH f AS (
            SELECT r.book_id,
                   AVG(r.rating)avg_rating,
                   COUNT(r.rating)
            FROM ratings r
            GROUP BY 1
            HAVING COUNT(rating) >= 50)
            
            SELECT a.author,
                   AVG(f.avg_rating)
            FROM f
            LEFT JOIN books b ON b.book_id = f.book_id
            LEFT JOIN authors a ON b.author_id = a.author_id
            GROUP BY 1
            ORDER BY 2 DESC
            LIMIT 1
        '''
pd.io.sql.read_sql(query, con = engine) 

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой средней оценкой книг, при учете книг с 50 и более оценками, это Джоан Кэтлин Роулинг, известная всем серией книг про Гарри Поттера.

**5 задание:** Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [8]:
query = '''
            WITH f AS (
            SELECT username,
                   COUNT(rating) rating_count
            FROM ratings 
            GROUP BY 1
            HAVING COUNT(rating) > 50),
            
            s AS (
            SELECT username, 
                   COUNT(text) review_count 
            FROM reviews
            GROUP BY 1)
            
            SELECT ROUND(AVG(review_count)) avg_review_count
            FROM f
            LEFT JOIN s ON f.username = s.username
        '''
pd.io.sql.read_sql(query, con = engine) 

,avg_review_count
0,24.0


В среднем, пользователи, которые поставили больше 50 оценок пишут 24 обзора на книги.

## <a id='the_destination_3'></a> ВЫВОДЫ:

- с 1 января 2000 года вышла 821 книга
- самое большое количество обзоров получила книга из серии "Сумерки" - 7 отзывов, хотя рейтинг не высокий. Так же в данных много книг вообще без обзоров
- исключив из данных брошюры (книги меньше 50 страниц), получили, что издательство "Penguin Books" выпустило больше всех книг, которые есть в нашей базе, а именно 42 шт
- автор с самой высокой средней оценкой книг, при учете книг с 50 и более оценками, это Джоан Кэтлин Роулинг, известная всем серией книг про Гарри Поттера
- в среднем, пользователи, которые поставили больше 50 оценок пишут 24 обзора на книги.

<a href='#the_destination_0'>К оглавлению</a>